In [1]:
# LIBRARIES
import snowflake.connector
import logging
from os import environ
from dotenv import load_dotenv
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
import optuna
import joblib
import numpy as np

In [2]:
# SNOWFLAKE CONNECTION 
load_dotenv()
for logger_name in ['snowflake.connector', 'botocore']:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    logger.addHandler(handler)

conn = snowflake.connector.connect(
    user = environ['USER'],
    password = environ['PASSWORD'],
    account = environ['ACCOUNT'],
    warehouse = environ['WAREHOUSE'],
    database = environ['DATABASE'],
    schema = environ['SCHEMA']
)

Snowflake Connector for Python Version: 3.12.3, Python Version: 3.10.14, Platform: Windows-10-10.0.22631-SP0
connect
__config
Connecting to GLOBAL Snowflake domain
This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
use_numpy: False
initialized
REST API object was created: fa62087.us-east-2.aws.snowflakecomputing.com:443
authenticate
account=fa62087, user=PAULO_MAIA, database=POKER_PROD, schema=POKER, warehouse=REPORTING_POKER_WH, role=None, request_id=9f8661fc-4203-41b4-9c51-3bae574d2f3b
body['data']: {'CLIENT_APP_ID': 'PythonConnector', 'CLIENT_APP_VERSION': '3.12.3', 'SVN_REVISION': None, 'ACCOUNT_NAME': 'fa62087', 'LOGIN_NAME': 'PAULO_MAIA', 'CLIENT_ENVIRONMENT': {'APPLICATION': 'PythonConnector', 'OS': 'Windows', 'OS_VERSION': 'Windows-10-10.0.22631-SP0', 'PYTHON_VERSION': '3.10.14', 'PYTHON_RUNTI

In [3]:
# SELECTING THE QUERY AND GETTING THE DATA
with open('training_query.sql', 'r') as file:
    query = file.read()
    
data = pd.read_sql_query(query, conn)

C:\Users\maiap\AppData\Local\Temp\ipykernel_11484\672369880.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)
cursor
executing SQL/command
query: [WITH counts AS ( SELECT start_date_local, start_hour_local, COUNT(*) AS same_hou...]
binding: [WITH counts AS ( SELECT start_date_local, start_hour_local, COUNT(*) AS same_hou...] with input=[None], processed=[{}]
sequence counter: 1
Request id: b60f6522-9849-4ab8-b077-f71a8e9ef532
running query [WITH counts AS ( SELECT start_date_local, start_hour_local, COUNT(*) AS same_hou...]
is_file_transfer: True
_cmd_query
serialize_to_dict() called
sql=[WITH counts AS ( SELECT start_date_local, start_hour_local, COUNT(*) AS same_hou...], sequence_id=[1], is_file_transfer=[False]
Opentelemtry otel injection failed because of: No module named 'opentelemetry'


In [4]:
# ENCODING CATEGORICAL COLUMNS AND DEFINING X AND Y
encoder = OneHotEncoder(sparse_output=False)
data_transform = data.drop(columns=['MATCH_ID', 'START_DATE_LOCAL'])
cat_columns = ['START_HOUR_LOCAL', 'DAY_OF_WEEK_LOCAL', 'KO_TYPE', 
               'MTT_POOL_ALLOCATION_DESCRIPTION', 'ESTRUTURA_BLINDS']
encoded_columns = encoder.fit_transform(data_transform[cat_columns])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(cat_columns))
data_final = pd.concat([data_transform.drop(cat_columns, axis=1), encoded_df], axis=1)

X = data_final.drop(columns=['HAS_OVERLAY', 'OVERLAY', 'MTT_RESULT', 'COLLECTED', 'NORMALIZED_MTT_RESULT', 'NORMALIZED_COLLECTED' ], axis=1)
y = data_final['COLLECTED']

In [6]:
X

,LEVEL_REJECT,MTT_BLIND_UP_TIME_SEC,LATE_REGISTRATION_TIME,EARLYBIRD,HAS_REBUY,HAS_ADDON,MTT_CUSTOMER_BUY_IN,MTT_CUSTOMER_REBUY,MTT_CUSTOMER_ADDON,GTD,...,MTT_POOL_ALLOCATION_DESCRIPTION_10% Plus,MTT_POOL_ALLOCATION_DESCRIPTION_15%,MTT_POOL_ALLOCATION_DESCRIPTION_15% Flat,MTT_POOL_ALLOCATION_DESCRIPTION_20%,MTT_POOL_ALLOCATION_DESCRIPTION_20% Flat,ESTRUTURA_BLINDS_Hyper,ESTRUTURA_BLINDS_Hyper - no ante,ESTRUTURA_BLINDS_Standard - no ante,ESTRUTURA_BLINDS_Turbo,ESTRUTURA_BLINDS_Turbo - no ante
0,13,360,4680.0,0.2,True,True,0.8,0.8,0.8,600,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,14,600,8400.0,0.2,True,False,5.0,5.0,NaN,1400,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,12,360,4320.0,0.2,True,True,0.2,0.2,0.2,60,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,13,420,5460.0,0.2,True,True,1.0,1.0,1.0,400,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,12,360,4320.0,0.2,True,True,0.2,0.2,0.2,100,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32913,14,720,10080.0,0.2,False,False,7.0,NaN,NaN,600,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
32914,14,720,10080.0,0.2,False,False,7.0,NaN,NaN,600,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
32915,15,300,4500.0,0.2,True,True,0.6,0.6,0.6,30,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32916,15,600,9000.0,0.2,True,True,3.0,3.0,4.0,4000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
# DEFINING THE CROSS-VALIDATION FUNCTION FOR XGBOOST REGRESSOR MODEL WITH OPTUNA
def cross_validation_optuna(trial, X, y, n_splits):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.8)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 5)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-5, 1e1, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 1e-5, 1e1, log=True)


    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma = gamma,
        min_child_weight = min_child_weight,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        device='cuda',
        random_state=42
    )

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    results_list = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        # CREATION OF A DATAFRAME TO SAVE ALL THE RESULTS
        results_df = pd.DataFrame({
            'Actual': y_test,
            'Prediction': predictions,
        })

        # ADDING ADDITIONAL METRICS TO THE DATAFRAME
        results_df['Absolute difference'] = np.abs(results_df['Actual'] - results_df['Prediction'])
        results_df['Mean square difference'] = (results_df['Actual'] - results_df['Prediction']) ** 2
        results_df['Error percentage'] = (results_df['Actual'] - results_df['Prediction']) / results_df['Actual'] * 100
        results_df['Absolute error percentage'] = np.abs(results_df['Error percentage'])

        results_list.append(results_df)

    full_results_df = pd.concat(results_list, ignore_index=True)

    return full_results_df, model


# DEFINING THE OPTIMIZATION FUNCTION FOR HYPERPARAMETER TUNING
def optimize_hyperparameters(X, y):
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())

    def objective(trial):
        full_results_df, model = cross_validation_optuna(trial, X, y, n_splits=10)
        trial.set_user_attr("model", model)
        trial.set_user_attr("results", full_results_df)
        return full_results_df['Absolute difference'].median()  # Using MSE as the optimization target

    study.optimize(objective, n_trials=40)

    print("Best hyperparameters: ", study.best_params)

    best_trial = study.best_trial
    best_model = best_trial.user_attrs["model"]
    best_results = best_trial.user_attrs["results"]

    return best_model, best_results

# RUNNING THE MODEL 
best_model, best_results_df = optimize_hyperparameters(X, y)

[I 2024-12-03 21:09:25,091] A new study created in memory with name: no-name-ed3712df-8cc7-47de-8d8e-e29bb7e079f5
C:\Users\maiap\anaconda3\envs\suprema-env\lib\site-packages\xgboost\core.py:158: UserWarning: [21:09:27] WARNING: D:\bld\xgboost-split_1732667012888\work\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-12-03 21:09:40,205] Trial 0 finished with value: 35.974033203124975 and parameters: {'n_estimators': 89, 'max_depth': 20, 'learning_rate': 0.7738115619023965, 'subsample': 0.5541628241200105, 'colsample_bytree': 0.5894367486593496, 'gamma': 0.9252685003610706, 'min_child_w

Best hyperparameters:  {'n_estimators': 189, 'max_depth': 14, 'learning_rate': 0.07668507588324075, 'subsample': 0.8580611031025586, 'colsample_bytree': 0.7750716058559984, 'gamma': 3.887818337250034, 'min_child_weight': 8, 'reg_alpha': 0.11559490931535855, 'reg_lambda': 0.002532190161340966}


In [7]:
# SAVING THE MODEL AND THE ENCODER
joblib.dump(encoder, '../new_model/encoder_v2.pkl')
joblib.dump(best_model, '../new_model/model_v2.pkl')

['../new_model/model_v2.pkl']